In [2]:
sealed trait Program
sealed trait Expr
case class Const(f: Double) extends Expr
case class Ident(s: String) extends Expr
case class Minus(e1: Expr, e2: Expr) extends Expr
case class Plus(e1: Expr, e2: Expr) extends Expr
case class Mult(e1: Expr, e2: Expr) extends Expr
case class Eq(e1: Expr, e2: Expr) extends Expr
case class Geq(e1: Expr, e2: Expr) extends Expr
case class IfThenElse(e1: Expr, e2: Expr, e3: Expr) extends Expr
case class Let(x: String, e1: Expr, e2: Expr) extends Expr
case class FunDef(id: String, e: Expr) extends Expr
case class FunCall(calledFun: Expr, argExpr: Expr) extends Expr
/* Here is the new stuff */
case class LetRec(funName: String, param: String, funExpr: Expr, bodyExpr: Expr) extends Expr

case class TopLevel(e: Expr) extends Program

defined trait Program
defined trait Expr
defined class Const
defined class Ident
defined class Minus
defined class Plus
defined class Mult
defined class Eq
defined class Geq
defined class IfThenElse
defined class Let
defined class FunDef
defined class FunCall
defined class LetRec
defined class TopLevel

In [4]:
sealed trait Environment 
sealed trait Value


/* -- We need to redefine values to accomodate the new representation of environments --*/
case class NumValue(d: Double) extends Value
case class BoolValue(b: Boolean) extends Value
case class Closure(x: String, e: Expr, pi: Environment) extends Value
case object ErrorValue extends Value


/*2. Operators on values */

def valueToNumber(v: Value): Double = v match {
    case NumValue(d) => d
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a number")
}

def valueToBoolean(v: Value): Boolean = v match {
    case BoolValue(b) => b
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a boolean")
}

def valueToClosure(v: Value): Closure = v match {
    case Closure(x, e, pi) => Closure(x, e, pi)
    case _ =>  throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a closure")
}


defined trait Environment
defined trait Value
defined class NumValue
defined class BoolValue
defined class Closure
defined object ErrorValue
defined function valueToNumber
defined function valueToBoolean
defined function valueToClosure

In [5]:
sealed trait Environment {
    def lookup(x:String): Value //define the signature 
}
case object EmptyEnv extends Environment  {
    //we could define inside the class or use case pattern matching and define it outside 
    override def lookup(x:String): Value = {
        throw new IllegalArgumentException(s"I know nothing about string $x")  
    }
}
case class Extend(x: String, v: Value, existing_env: Environment) extends Environment   {
    override def lookup(what:String): Value = {
        if (what == x) { v }
        else {
            existing_env.lookup(what)
        }
    }
}
case class ExtendRec(recfuncname: String, formalparam: String, body: Expr, existing_env: Environment) extends Environment {
    override def lookup(what: String): Value = {
        if (what!= recfuncname) {
            existing_env.lookup(what)
        }
        else {
            Closure(formalparam, body, existing_env)
        }
    }
}

cmd5.sc:24: type mismatch;
 found   : Environment (in ammonite.$sess.cmd5.Helper) 
 required: Environment (in ammonite.$sess.cmd3.Helper) 
            Closure(formalparam, body, existing_env)
                                       ^Compilation Failed

: 

In [9]:
val empty= EmptyEnv
//tacking on to the emptyenviron
val env1= Extend("x", NumValue(10), empty)
//extending an existing environment
val env2= Extend("y", NumValue(20), env1)

empty: EmptyEnv.type = EmptyEnv
env1: Extend = Extend(x = "x", v = NumValue(d = 10.0), existing_env = EmptyEnv)
env2: Extend = Extend(
  x = "y",
  v = NumValue(d = 20.0),
  existing_env = Extend(
    x = "x",
    v = NumValue(d = 10.0),
    existing_env = EmptyEnv
  )
)

cmd9.sc:7: type mismatch;
 found   : Helper.this.ExtendRec
 required: Map[String,cmd9.this.cmd1.Value]
            Closure(formalparam, body,this)
                                      ^Compilation Failed

: 